# Notebook Containing the data generation, NN Modelling and Optimization Attempts for two windturbines

In [6]:
import pandas as pd
import matplotlib.pyplot as plt

# Import Pyomo
from pyomo import environ
from pyomo.core import *
import pyomo.environ as pyo

# import constraint Learning Tool
import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), 'DistCL_code'))
from distcl import distcl

## Load Data

In [7]:
data = pd.read_csv("data/two_windturbine.csv")
data.head()

,x_turb2,y_turb2,wind_speed,wind_direction,turbulence_intensity,turbine1_power,turbine2_powers,farm_power
0,800,0,8.0,270.0,0.06,1753.954459,615.070589,2369.025048
1,800,200,8.0,270.0,0.06,1753.954459,1751.593411,3505.547870
2,800,400,8.0,270.0,0.06,1753.954459,1753.954459,3507.908918
3,800,600,8.0,270.0,0.06,1753.954459,1753.954459,3507.908918
4,900,0,8.0,270.0,0.06,1753.954459,694.023084,2447.977544


# Train NN and generate constraints

In [8]:
### Test/Train Split
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

val_ind = val_data.index
test_ind = test_data.index

In [17]:
# Retrain the model with the best parameters
cl_tool = distcl(X=data[["x_turb2", "y_turb2", "wind_speed", "wind_direction"]],#, "turbulence_intensity"]],
            y=data["farm_power"], n_preds=1, val_ind=val_ind, test_ind=test_ind)

model, preds_test, sd_test, y_test = cl_tool.train(n_hidden=5, n_nodes=40, iters=5000, drop=0.05, learning_rate=1e-4)

DistFCNN(
  (lin_layers): ModuleList(
    (0): Linear(in_features=4, out_features=40, bias=True)
    (1-4): 4 x Linear(in_features=40, out_features=40, bias=True)
  )
  (output_mean_layer): Linear(in_features=40, out_features=1, bias=True)
  (output_sd_layer): Linear(in_features=40, out_features=1, bias=True)
  (droput_layers): ModuleList(
    (0-4): 5 x Dropout(p=0.05, inplace=False)
  )
)
cpu
epoch 0 loss 264480.25
epoch 500 loss -0.06833779811859131
epoch 1000 loss -0.07717901468276978
epoch 1500 loss -0.3067978620529175
epoch 2000 loss -0.416925311088562
epoch 2500 loss -0.4519554078578949
epoch 3000 loss -0.49979719519615173
epoch 3500 loss -0.5331390500068665
epoch 4000 loss -0.5621916651725769
epoch 4500 loss -0.5932419300079346
NN fitting process finished with a validation GAUSSIANNLL loss of -0.6084268689155579 in epoch 4821


In [18]:
cons = cl_tool.constraint_build(model)
cons.to_csv('inputs/constraints_twoturbines.csv')
cons = pd.read_csv('inputs/constraints_twoturbines.csv', index_col=0)
cons

,intercept,layer,node,node_0,node_1,node_2,node_3,node_4,node_5,node_6,...,node_30,node_31,node_32,node_33,node_34,node_35,node_36,node_37,node_38,node_39
0,-0.062445,0,0,-0.003743,0.268222,-0.435184,-0.417960,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.066713,0,1,-0.192577,0.134079,0.006372,0.426913,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.448675,0,2,-0.044372,0.132306,-0.151107,-0.098283,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.431719,0,3,-0.477674,-0.331141,-0.206112,0.018522,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.034038,0,4,0.197668,0.300011,-0.380301,-0.270956,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,-0.146444,4,37,0.080736,-0.011154,-0.157972,-0.048303,-0.054471,0.008119,-0.047746,...,0.011753,0.136135,0.110910,0.047506,-0.000935,0.055057,0.035651,-0.014222,0.124723,0.142872
38,0.054022,4,38,-0.097866,0.146437,-0.040027,-0.073604,0.142411,-0.071325,0.069713,...,-0.037923,0.047731,-0.157295,0.020190,-0.089610,0.011639,0.097862,0.131261,-0.034903,0.167303
39,-0.026529,4,39,-0.002430,-0.101473,0.073348,-0.041979,-0.119073,0.106021,0.159447,...,0.066780,0.019991,-0.069851,-0.061313,0.096257,-0.129365,-0.047544,-0.153952,-0.085181,0.011133
0,0.115418,5,0,0.099472,0.064389,0.116247,0.011643,-0.023010,0.056138,0.105994,...,-0.143681,0.030618,0.054342,0.171972,0.138181,-0.077397,0.044713,0.014941,0.077141,-0.094754


# Optimization Model



### Decision Variables
\( x \): x-coordinate of the second turbine relative to the first.
\( y \): y-coordinate of the second turbine relative to the first.

### **Objective Function**
$$ \max_{x, y} P(x, y) $$

### Constraints
$ x_{\min} \leq x \leq x_{\max} $
   
$ y_{\min} \leq y \leq y_{\max} $


In [21]:
contextual_sample

,x_turb2,y_turb2,wind_speed,wind_direction
0,800,0,8.0,270.0
1,800,200,8.0,270.0
2,800,400,8.0,270.0
3,800,600,8.0,270.0
4,900,0,8.0,270.0
...,...,...,...,...
163,4800,600,8.0,270.0
164,4900,0,8.0,270.0
165,4900,200,8.0,270.0
166,4900,400,8.0,270.0


In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, minimize, value


contextual_sample = data[["x_turb2", "y_turb2", "wind_speed", "wind_direction"]]#, "turbulence_intensity"]]



### Define the optimization model
model = ConcreteModel()


model.N = pyo.RangeSet(0,1)
model.var_ind = pyo.Set(initialize=contextual_sample.columns.sort_values())
model.x = pyo.Var(model.var_ind, model.N,  within=pyo.Reals) # first stage decisions about price and contextual variables
# TODO: does this make any sense ? 



# Define decision variables for the x and y coordinates of turbine 2
model.x_turb2 = Var(within=PositiveReals, bounds=(800, 5000))
model.y_turb2 = Var(within=PositiveReals, bounds=(0, 600))

model.y = pyo.Var(pyo.Any, dense=False, domain=pyo.Reals) # learned variables (demand)

model.power = pyo.Var(within=pyo.Reals) # saving profit per scenario


## obj function 
#definition of the objective function
def obj_expression(model):
    return model.power
model.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

#power generation
def power(model):
    return model.power ==  model.y['power']
model.const_power = pyo.Constraint(rule=power)


# ## min distance 
# # Define constraints if any (for example, minimum distance between turbines)
# def distance_constraint(model):
#     return ((model.x_turb2 - 0)**2 + (model.y_turb2 - 0)**2) >= 400**2 
# model.distance_constraint = Constraint(rule=distance_constraint)




### Constraint Embedding
cl_tool.const_embed(opt_model=model, constaints=cons, outcome='power', deterministic = True)


# Solve the optimization problem
solver = SolverFactory('glpk')
solver.options['threads'] = 8
solver.options['NonConvex'] = 2
results = solver.solve(model, tee=True)




AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [25]:
print(SolverFactory('gurobi').available())

False


In [ ]:


# Get the optimized coordinates and farm power
optimized_x_turb2 = value(model.x_turb2)
optimized_y_turb2 = value(model.y_turb2)
turbine_powers, optimized_farm_power = two_turbine_simulation(fmodel, 
                                                              x_turb2=optimized_x_turb2, 
                                                              y_turb2=optimized_y_turb2,
                                                              wind_speeds=wind_speeds, 
                                                              wind_directions=wind_directions, 
                                                              turbulence_intensities=turbulence_intensities)

print(f"Optimized x coordinate of turbine 2: {optimized_x_turb2}")
print(f"Optimized y coordinate of turbine 2: {optimized_y_turb2}")
print(f"Optimized farm power: {optimized_farm_power[0]}")